In [3]:
import os
import json
import pandas as pd
import numpy as np
import psycopg2
import joblib
import logging
from typing import List
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
from sklearn.neighbors import LocalOutlierFactor
import logging
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    r2_score,
    explained_variance_score,
    mean_absolute_error,
    mean_squared_error
)
from datetime import datetime, timedelta
import torch
import torch.nn as nn
from typing import Optional
import duckdb

import re
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
# Set options to show full DataFrame output
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#conn = psycopg2.connect(host='fortifai-ng-dev-db.postgres.database.azure.com',
#			database='baldota-dev-db',
#			user='fortifai_ng_ai_user_rw',
#			password='AIPwd@123!',
#			port='5432',
#            sslmode="require"
#		)
#cur = conn.cursor()
#cur.execute("""
#            SELECT table_name 
#            FROM information_schema.tables 
#        """)

#tables = [row[0] for row in cur.fetchall()]
#po_data= pd.read_sql_query(f"SELECT * FROM staging_db.{'po_header_lineitem_merged_with_risks'}", conn)
#cur.close()
#conn.close()
po_data=pd.read_excel(r"C:\Users\ajayn\Downloads\SARA-23-09\New 7 Rules\pipeline\data_for_db_update_without_risk_driver_17_10.xlsx")
#p2p_header_po_data.head()
print(po_data.shape)
#po_data.head()
po_data['net_val_po_curr_src_po_with_exchange_rate']=po_data['net_val_po_curr_src_po']*po_data['exchange_rate_hpd_po']

# Example: assuming your date column is named 'po_date'
#po_data['po_date'] = pd.to_datetime(po_data['po_date'], errors='coerce')

# Calculate the cutoff date (today - 6 months)
cutoff_date = pd.Timestamp.today() - pd.DateOffset(months=6)

# Filter rows where po_date is within last 6 months
po_last_6m = po_data[po_data['purch_doc_date_hpd_po'] >= cutoff_date].copy()

print(f"Filtered {len(po_last_6m)} records from last 6 months out of {len(po_data)} total.")


conn = psycopg2.connect(host='fortifai-ng-dev-db.postgres.database.azure.com',
			database='baldota-dev-db',
			user='fortifai_ng_ai_user_rw',
			password='AIPwd@123!',
			port='5432',
            sslmode="require"
		)
cur = conn.cursor()
cur.execute("""
            SELECT table_name 
            FROM information_schema.tables 
        """)

tables = [row[0] for row in cur.fetchall()]
fp_data= pd.read_sql_query(f"SELECT * FROM admin_db.{'false_positive_criteria'}", conn)
cur.close()
conn.close()
#p2p_header_po_data.head()
print(fp_data.shape)


# register your two DataFrames as in-memory tables
duckdb.register('po', po_last_6m)
duckdb.register('fp', fp_data)
query = """
SELECT *
FROM po AS t
WHERE NOT EXISTS (
    SELECT 1
    FROM fp
    WHERE (
        -- Supplier-based exclusion
        UPPER(CAST(fp.entity_type AS VARCHAR)) = 'SUPPLIER'
        AND UPPER(CAST(fp.entity_id AS VARCHAR)) = UPPER(CAST(t.vendor_or_creditor_acct_no_hpd_po AS VARCHAR))
        AND CASE
            WHEN TRIM(fp.operator) = '<=' THEN t.net_val_po_curr_src_po_with_exchange_rate <= CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '<'  THEN t.net_val_po_curr_src_po_with_exchange_rate  < CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '>=' THEN t.net_val_po_curr_src_po_with_exchange_rate >= CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '>'  THEN t.net_val_po_curr_src_po_with_exchange_rate  > CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '='  THEN t.net_val_po_curr_src_po_with_exchange_rate  = CAST(fp.threshold_value AS DOUBLE)
            ELSE FALSE
        END
    )
    OR (
        -- Material-based exclusion
        UPPER(CAST(fp.entity_type AS VARCHAR)) = 'MATERIAL'
        AND UPPER(CAST(fp.entity_id AS VARCHAR)) = UPPER(CAST(t.material_no_src_po AS VARCHAR))
        AND CASE
            WHEN TRIM(fp.operator) = '<=' THEN t.net_val_po_curr_src_po_with_exchange_rate <= CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '<'  THEN t.net_val_po_curr_src_po_with_exchange_rate  < CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '>=' THEN t.net_val_po_curr_src_po_with_exchange_rate >= CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '>'  THEN t.net_val_po_curr_src_po_with_exchange_rate  > CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '='  THEN t.net_val_po_curr_src_po_with_exchange_rate  = CAST(fp.threshold_value AS DOUBLE)
            ELSE FALSE
        END
    )
    OR (
        -- Global transaction exclusion
        UPPER(CAST(fp.entity_type AS VARCHAR)) = 'TRANSACTION'
        AND CASE
            WHEN TRIM(fp.operator) = '<=' THEN t.net_val_po_curr_src_po_with_exchange_rate <= CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '<'  THEN t.net_val_po_curr_src_po_with_exchange_rate  < CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '>=' THEN t.net_val_po_curr_src_po_with_exchange_rate >= CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '>'  THEN t.net_val_po_curr_src_po_with_exchange_rate  > CAST(fp.threshold_value AS DOUBLE)
            WHEN TRIM(fp.operator) = '='  THEN t.net_val_po_curr_src_po_with_exchange_rate  = CAST(fp.threshold_value AS DOUBLE)
            ELSE FALSE
        END
    )
)
"""


cleaned = duckdb.sql(query).to_df()
print(f"Filtered: {len(cleaned)} rows kept (out of {len(po_last_6m)})")


# ============================================================
# FortifAI — Reports + Scheduler (Local-first; Blob optional)
# - You pass a ready DataFrame: `cleaned_df`
# - Generates Daily/Weekly/Monthly reports
# - Saves locally; uploads to Azure Blob only if creds provided
# - Public API:
#     run_reports_job(cleaned_df, mode="daily"|"weekly"|"monthly"|"backfill6m",
#                     upload_to_blob=None)
#     start_report_scheduler(get_df_callable)
# ============================================================

# ------------------------------
# CONFIG
# ------------------------------
# ============================================================
# FortifAI — Batch Reports (Daily / Weekly / Monthly) + Backfill
# Exact schemas + ALL Sub-Risk (1..3) and Impact Area (1..5)
# ============================================================

from __future__ import annotations

import os
import re
from datetime import date, timedelta
from typing import Optional

import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

# ------------------------------
# Config
# ------------------------------
BASE_OUT     = "generate_reports"
TZ_NAME      = "Asia/Kolkata"
RUN_DATE     = pd.Timestamp.now(tz=TZ_NAME).date()
DATE_COL     = None  # set if you know the exact date column

# Toggle specific report families
RUN_TOP10_VENDORS_AND_TXNS = True
RUN_HIGHRISK_TXNS          = True
RUN_SPLIT_PO               = True
RUN_PRICE_VARIANCE         = True

# ------------------------------
# Excel engine & formatting utils
# ------------------------------
def pick_engine():
    try:
        import xlsxwriter  # noqa: F401
        return "xlsxwriter"
    except Exception:
        return "openpyxl"

ENGINE = pick_engine()

def writer_with_formats(path):
    w = pd.ExcelWriter(path, engine=ENGINE)
    if ENGINE == "xlsxwriter":
        wb = w.book
        fmt = {
            "money": wb.add_format({"num_format": "#,##0.00"}),
            "text":  wb.add_format({"num_format": "@"}),
        }
        return w, fmt
    return w, None

def autofit_and_formats(writer, sheet_name, df, fmt):
    money_like = {
        "value", "impact value", "impact_value", "total_value", "total_impact", "Value",
        "price per unit (doc curr)", "net po value (doc curr)",
        "gross po value (doc curr)", "total value on release"
    }
    if ENGINE == "xlsxwriter":
        ws = writer.sheets[sheet_name]
        ws.freeze_panes(1, 0)
        ws.autofilter(0, 0, len(df), max(len(df.columns)-1, 0))
        for i, c in enumerate(df.columns):
            cl = c.lower()
            if c in {"document_id", "line_item", "cost_center"}:
                ws.set_column(i, i, 20, fmt["text"])
            elif cl in money_like:
                ws.set_column(i, i, 18, fmt["money"])
            elif cl in {"vendor", "vendor name", "vendor_name"}:
                ws.set_column(i, i, 34)
            else:
                ws.set_column(i, i, 18)
    else:
        from openpyxl.utils import get_column_letter
        ws = writer.sheets[sheet_name]
        ws.freeze_panes = "A2"
        ws.auto_filter.ref = ws.dimensions
        for i, _ in enumerate(df.columns, start=1):
            ws.column_dimensions[get_column_letter(i)].width = 18

# ------------------------------
# Date column detection
# ------------------------------
DATE_CANDIDATES = [
    'purch_doc_date_hpd_po',  # present in your schema
    'po_date',
    'document_date',
]

def choose_date_col(df):
    if DATE_COL and DATE_COL in df.columns:
        return DATE_COL
    for c in DATE_CANDIDATES:
        if c in df.columns:
            try:
                as_dt = pd.to_datetime(df[c], errors="coerce")
                if as_dt.notna().any():
                    return c
            except Exception:
                pass
    raise ValueError("No usable date column found. Set DATE_COL or add to DATE_CANDIDATES.")

# ------------------------------
# Period helpers
# ------------------------------
def clamp_to_six_months(df, dcol):
    six_months_ago = RUN_DATE - relativedelta(months=6)
    data_min_ts = pd.to_datetime(df[dcol], errors="coerce")
    data_min = data_min_ts.min().date() if data_min_ts.notna().any() else RUN_DATE
    start = max(six_months_ago, data_min)
    return start, RUN_DATE

def iter_daily(start: date, end: date):
    d = start
    while d <= end:
        yield d, d
        d += timedelta(days=1)

def start_of_week_monday(d: date):
    return d - timedelta(days=d.weekday())  # Monday=0

def iter_weekly(start: date, end: date):
    w_start = start_of_week_monday(start)
    while w_start <= end:
        w_end = min(w_start + timedelta(days=6), end)
        s = max(w_start, start)
        yield s, w_end
        w_start += timedelta(days=7)

def iter_monthly(start: date, end: date):
    m_start = date(start.year, start.month, 1)
    while m_start <= end:
        next_month = (m_start + relativedelta(months=1))
        m_end = min(next_month - timedelta(days=1), end)
        s = max(m_start, start)
        yield s, m_end
        m_start = next_month

# ============================================================
# P2P detail mapping (tailored to your columns) — now with ALL sub-risks & impacts
# ============================================================
def _first_present(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

# Output label -> source priority list (first present is used)
P2P_MAP = {
    "PO Number":                     ["purch_doc_no_src_po", "base_id_src_po"],
    "PO Item Number":                ["purch_doc_item_no_src_po"],
    "PO Creation Date":              ["purch_doc_date_hpd_po"],
    "PO Category":                   ["purch_doc_category_hpd_po"],
    "Vendor Account Number":         ["vendor_or_creditor_acct_no_hpd_po", "vendor_or_creditor_acct_no"],
    "Vendor Name":                   ["vendor_name_1"],

    "Material Number":               ["material_no_src_po"],
    "Short Text / Description":      ["short_text_src_po"],
    "Material Type":                 ["material_type_src_po"],
    "Material Group":                ["matl_group_src_po"],

    "Order UoM":                     ["order_uom_src_po"],
    "Ordered Quantity":              ["quantity_src_po"],
    "Plant Code":                    ["plant_src_po"],
    "Company Code":                  ["company_code_src_po"],

    "Price per Unit (Doc Curr)":     ["net_price_doc_curr_src_po"],
    "Net PO Value (Doc Curr)":       ["net_val_po_curr_src_po"],

    "PR Number":                     ["pr_no_src_po"],
    "PR Item Number":                ["pr_item_no_src_po"],
    "PO Requester Name":             ["requester_name_src_po"],

    "PO Item Deletion Flag":         ["po_item_del_flag_src_po"],
    "GR Indicator":                  ["gr_indicator_src_po"],
    "Gross PO Value (Doc Curr)":     ["gross_val_po_curr_src_po"],
    "Invoice Receipt Indicator":     ["inv_receipt_indicator_src_po", "gr_invoice_verif_flag_src_po"],

    "Company Code (Header)":         ["company_code_hpd_po"],
    "Currency":                      ["currency_hpd_po"],
    "Exchange Rate":                 ["exchange_rate_hpd_po"],

    "Created By":                    ["object_created_by_hpd_po", "object_created_by"],
    "Principal Purchase Agreement No.": ["principal_purch_agrmt_no_hpd_po"],
    "Purchase Group":                ["purch_group_hpd_po"],
    "Purchasing Organization":       ["purch_org_hpd_po"],
    "Payment Terms":                 ["pymnt_terms_hpd_po"],
    "PO Processing Status":          ["processing_status_hpd_po"],
    "Release Indicator":             ["release_indicator_hpd_po"],
    "Release Status":                ["release_status_hpd_po"],
    "Responsible Vendor / Salesperson": ["resp_vendor_salesperson_hpd_po"],

    "Total Value on Release":        ["on_release_total_value_hpd_po"],
    "PO Type":                       ["purch_doc_type_hpd_po"],

    "Risk Score":                    ["risk_score", "gbt_fraud_score", "ae_fraud_score", "iso_fraud_score"],
    "Risk Level":                    ["risk_level"],
    "Risk Scenario":                 ["main_risk_scenario"],

    # ALL Sub-Risks
    "Sub-Risk 1":                    ["sub_risk_1"],
    "Sub-Risk 2":                    ["sub_risk_2"],
    "Sub-Risk 3":                    ["sub_risk_3"],

    # ALL Impact Areas
    "Impact Area 1":                 ["impact_1"],
    "Impact Area 2":                 ["impact_2"],
    "Impact Area 3":                 ["impact_3"],
    "Impact Area 4":                 ["impact_4"],
    "Impact Area 5":                 ["impact_5"],
}
P2P_ORDER = list(P2P_MAP.keys())

def make_p2p_detail(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    rename_map = {}
    for out_col, candidates in P2P_MAP.items():
        src = _first_present(df, candidates)
        if src is not None:
            rename_map[src] = out_col
    if not rename_map:
        return pd.DataFrame(columns=P2P_ORDER)

    out = df.rename(columns=rename_map)

    # Coerce numerics for money/qty/score
    for col in ["Price per Unit (Doc Curr)", "Net PO Value (Doc Curr)",
                "Gross PO Value (Doc Curr)", "Total Value on Release",
                "Ordered Quantity", "Risk Score"]:
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors="coerce")

    # Stringify key identifiers
    for col in ["PO Number", "PO Item Number", "Vendor Account Number", "PR Number", "PR Item Number", "Plant Code"]:
        if col in out.columns:
            out[col] = out[col].astype("string")

    have = [c for c in P2P_ORDER if c in out.columns]
    return out.loc[:, have]

# ============================================================
# REPORT 1: Top 10 Vendors & Risky Transactions (exact schema)
#   - Sheet 1: Top10_Vendors with EXACT columns
#   - Sheet 2: Risky_Txns_Top10 with FULL P2P + all sub-risks/impacts
# ============================================================
def report_top10_vendors_and_txns(df_in, out_path):
    C = {
        "vendor_id":   "vendor_or_creditor_acct_no_hpd_po",
        "vendor_name": "vendor_name_1",
        "document_id": "purch_doc_no_src_po",
        "risk_level":  "risk_level",
        "plant":       "plant_src_po",
        "total_value": "on_release_total_value_hpd_po",
        "impact_value":"net_val_po_curr_src_po",
        "main_scenario":"main_risk_scenario",
        "process":     "process",
        "vendor_category": "vendor_category",
        "del_flag":    "po_item_del_flag_src_po",  # treat 'L' as deletion
    }
    FILTER_TO_P2P = True

    df = df_in.copy()
    present = {k: v for k, v in C.items() if v in df.columns}
    df = df.rename(columns={present[k]: k for k in present})

    for opt in ["vendor_category","impact_value","process","main_scenario","del_flag","plant","document_id","total_value","vendor_name"]:
        if opt not in df.columns: df[opt] = pd.NA
    df = df[~df["del_flag"].fillna("").astype(str).str.upper().eq("L")]

    if FILTER_TO_P2P:
        if "process" in df and df["process"].notna().any():
            df = df[df["process"].fillna("").str.upper().eq("P2P")]
        elif "main_scenario" in df and df["main_scenario"].notna().any():
            df = df[df["main_scenario"].fillna("").str.lower().eq("procurement risk")]

    risk_series = df["risk_level"].fillna("").astype(str).str.strip()
    df_risky = df[(~risk_series.str.lower().eq("no risk")) & (risk_series != "")]
    if df_risky.empty: return False

    df_risky["impact_value"] = pd.to_numeric(df_risky["impact_value"], errors="coerce").fillna(0.0)
    df_risky["total_value"]  = pd.to_numeric(df_risky["total_value"],  errors="coerce").fillna(0.0)

    # Vendor ranking: rows desc, then total impact desc
    grp = (
        df_risky.groupby(["vendor_id","vendor_name","vendor_category"], dropna=False)
                .agg(rows=("document_id","count"),
                     total_impact=("impact_value","sum"))
                .reset_index()
                .sort_values(["rows","total_impact"], ascending=[False, False])
                .head(10)
    )
    if grp.empty: return False

    top_vendor_ids = set(grp["vendor_id"].astype(str))

    # Sheet 1: EXACT columns
    idx = (
        df_risky[df_risky["vendor_id"].astype(str).isin(top_vendor_ids)]
        .sort_values(["vendor_id","impact_value"], ascending=[True, False])
        .groupby("vendor_id", as_index=False)
        .nth(0)
    )
    top10_table = (
        idx.rename(columns={
            "vendor_id": "Vendor",
            "vendor_category": "Vendor Category",
            "document_id": "Document ID",
            "risk_level": "Risk Level",
            "plant": "Plant",
            "total_value": "Total Value",
        })[["Vendor", "Vendor Category", "Document ID", "Risk Level", "Plant", "Total Value"]]
        .reset_index(drop=True)
    )

    # Sheet 2: FULL P2P + ALL sub-risks/impacts
    detail_base = df_risky[df_risky["vendor_id"].astype(str).isin(top_vendor_ids)].copy()
    detail = make_p2p_detail(detail_base)
    sort_cols = [c for c in ["Vendor Account Number","Vendor Name","Net PO Value (Doc Curr)"] if c in detail.columns]
    if sort_cols:
        detail = detail.sort_values(sort_cols, ascending=[True, True, False]).reset_index(drop=True)

    writer, fmt = writer_with_formats(out_path)
    top10_table.to_excel(writer, "Top10_Vendors", index=False)
    autofit_and_formats(writer, "Top10_Vendors", top10_table, fmt)
    detail.to_excel(writer, "Risky_Txns_Top10", index=False)
    autofit_and_formats(writer, "Risky_Txns_Top10", detail, fmt)
    writer.close()
    return True

# ============================================================
# REPORT 2: Top 10 High-Risk Transactions (Top10 + All) — FULL P2P + ALL sub-risks/impacts
# ============================================================
def report_highrisk_txns(df_in, out_path):
    RISKY_LEVELS = {"High Risk", "Very High Risk", "Needs Validation"}
    FILTER_TO_P2P = True

    df = df_in.copy()
    for opt in ["process","main_scenario","del_flag"]:
        if opt not in df.columns: df[opt] = pd.NA
    df = df[~df["del_flag"].fillna("").astype(str).str.upper().eq("L")]

    if FILTER_TO_P2P:
        if "process" in df and df["process"].notna().any():
            df = df[df["process"].fillna("").str.upper().eq("P2P")]
        elif "main_scenario" in df and df["main_scenario"].notna().any():
            df = df[df["main_scenario"].fillna("").str.lower().eq("procurement risk")]

    risk_series = df.get("risk_level", pd.Series(dtype=str)).fillna("").astype(str).str.strip()
    df_hr = df[risk_series.str.lower().isin({x.lower() for x in RISKY_LEVELS})]
    if df_hr.empty: return False

    detail_all = make_p2p_detail(df_hr)
    if detail_all.empty: return False

    sort_cols = [c for c in ["Net PO Value (Doc Curr)","Gross PO Value (Doc Curr)","Total Value on Release"] if c in detail_all.columns]
    detail_top10 = (detail_all.sort_values(sort_cols, ascending=[False]*len(sort_cols))
                               .head(10).reset_index(drop=True)) if sort_cols else detail_all.head(10)

    writer, fmt = writer_with_formats(out_path)
    detail_top10.to_excel(writer, "Top10_HighRisk_Txns", index=False)
    autofit_and_formats(writer, "Top10_HighRisk_Txns", detail_top10, fmt)
    detail_all.to_excel(writer, "All_HighRisk_Txns", index=False)
    autofit_and_formats(writer, "All_HighRisk_Txns", detail_all, fmt)
    writer.close()
    return True

# ============================================================
# REPORT 3: Split PO — FULL P2P + ALL sub-risks/impacts
# ============================================================
def report_split_po(df_in, out_path):
    FILTER_TO_P2P = True
    split_pattern = re.compile(r"\bsplit[\s_-]*po\b", flags=re.IGNORECASE)

    df = df_in.copy()
    for opt in ["process","main_scenario","del_flag","sub_risk_1","sub_risk_2","sub_risk_3"]:
        if opt not in df.columns: df[opt] = pd.NA
    df = df[~df["del_flag"].fillna("").astype(str).str.upper().eq("L")]

    if FILTER_TO_P2P:
        if "process" in df and df["process"].notna().any():
            df = df[df["process"].fillna("").str.upper().eq("P2P")]
        elif "main_scenario" in df and df["main_scenario"].notna().any():
            df = df[df["main_scenario"].fillna("").str.lower().eq("procurement risk")]

    def _is_split(val):
        if pd.isna(val): return False
        if isinstance(val, (list, tuple, set)):
            return any(bool(split_pattern.search(str(x))) for x in val)
        return bool(split_pattern.search(str(val)))

    risk_cols = [c for c in ["sub_risk_1","sub_risk_2","sub_risk_3"] if c in df.columns]
    if not risk_cols: return False

    mask = False
    for c in risk_cols:
        mask = mask | df[c].apply(_is_split)
    df_split = df[mask]
    if df_split.empty: return False

    detail = make_p2p_detail(df_split)
    if detail.empty: return False
    sort_cols = [c for c in ["Net PO Value (Doc Curr)","Gross PO Value (Doc Curr)","Total Value on Release"] if c in detail.columns]
    if sort_cols:
        detail = detail.sort_values(sort_cols, ascending=[False]*len(sort_cols))

    writer, fmt = writer_with_formats(out_path)
    detail.to_excel(writer, "SplitPO_Txns", index=False)
    autofit_and_formats(writer, "SplitPO_Txns", detail, fmt)
    writer.close()
    return True

# ============================================================
# REPORT 4: Price Variance — FULL P2P + ALL sub-risks/impacts
# ============================================================
def report_price_variance(df_in, out_path):
    FILTER_TO_P2P = True
    pv_pattern = re.compile(r"\bprice[\s_-]*variance\b", flags=re.IGNORECASE)

    df = df_in.copy()
    for opt in ["process","main_scenario","del_flag","sub_risk_1","sub_risk_2","sub_risk_3"]:
        if opt not in df.columns: df[opt] = pd.NA
    df = df[~df["del_flag"].fillna("").astype(str).str.upper().eq("L")]

    if FILTER_TO_P2P:
        if "process" in df and df["process"].notna().any():
            df = df[df["process"].fillna("").str.upper().eq("P2P")]
        elif "main_scenario" in df and df["main_scenario"].notna().any():
            df = df[df["main_scenario"].fillna("").str.lower().eq("procurement risk")]

    def _is_pv(val):
        if pd.isna(val): return False
        if isinstance(val, (list, tuple, set)):
            return any(bool(pv_pattern.search(str(x))) for x in val)
        return bool(pv_pattern.search(str(val)))

    risk_cols = [c for c in ["sub_risk_1","sub_risk_2","sub_risk_3"] if c in df.columns]
    if not risk_cols: return False

    mask = False
    for c in risk_cols:
        mask = mask | df[c].apply(_is_pv)
    df_pv = df[mask]
    if df_pv.empty: return False

    detail = make_p2p_detail(df_pv)
    if detail.empty: return False
    sort_cols = [c for c in ["Net PO Value (Doc Curr)","Gross PO Value (Doc Curr)","Total Value on Release"] if c in detail.columns]
    if sort_cols:
        detail = detail.sort_values(sort_cols, ascending=[False]*len(sort_cols))

    writer, fmt = writer_with_formats(out_path)
    detail.to_excel(writer, "PriceVariance_Txns", index=False)
    autofit_and_formats(writer, "PriceVariance_Txns", detail, fmt)
    writer.close()
    return True

# ============================================================
# RUNNER (folderized: Daily/Weekly/Monthly subfolders)
# ============================================================
def ensure_base_dirs():
    for sub in ["Daily", "Weekly", "Monthly"]:
        os.makedirs(os.path.join(BASE_OUT, sub), exist_ok=True)

def weekly_folder_name(start_d: date, end_d: date):
    iso_year, iso_week, _ = start_d.isocalendar()
    return f"{iso_year}-W{iso_week:02d}_{start_d:%Y-%m-%d}_to_{end_d:%Y-%m-%d}"

def monthly_folder_name(start_d: date, end_d: date):
    return f"{start_d:%Y-%m}"

def run_for_period(df_full, start_d: date, end_d: date, granularity: str):
    mask = (df_full["__report_date"].dt.date >= start_d) & (df_full["__report_date"].dt.date <= end_d)
    df_slice = df_full.loc[mask].copy()
    if df_slice.empty: return

    if granularity == "Daily":
        subfolder = f"{start_d:%Y-%m-%d}"
    elif granularity == "Weekly":
        subfolder = weekly_folder_name(start_d, end_d)
    elif granularity == "Monthly":
        subfolder = monthly_folder_name(start_d, end_d)
    else:
        subfolder = f"{start_d:%Y-%m-%d}_to_{end_d:%Y-%m-%d}"

    folder = os.path.join(BASE_OUT, granularity, subfolder)
    os.makedirs(folder, exist_ok=True)

    if RUN_TOP10_VENDORS_AND_TXNS:
        path = os.path.join(folder, "Top 10 high-risk vendors by risky transactions.xlsx")
        ok = report_top10_vendors_and_txns(df_slice, path)
        if ok: print(f"[{granularity}] {subfolder} → Top10 Vendors ✔")

    if RUN_HIGHRISK_TXNS:
        path = os.path.join(folder, "Top 10 high-risk transactions.xlsx")
        ok = report_highrisk_txns(df_slice, path)
        if ok: print(f"[{granularity}] {subfolder} → High-Risk Txns ✔")

    if RUN_SPLIT_PO:
        path = os.path.join(folder, "Split PO Report.xlsx")
        ok = report_split_po(df_slice, path)
        if ok: print(f"[{granularity}] {subfolder} → Split PO ✔")

    if RUN_PRICE_VARIANCE:
        path = os.path.join(folder, "Price Variance Risk Report.xlsx")
        ok = report_price_variance(df_slice, path)
        if ok: print(f"[{granularity}] {subfolder} → Price Variance ✔")

# ============================================================
# OPTIONAL BLOB UPLOAD (auto-disabled if creds missing)
# ============================================================
try:
    from azure.storage.blob import BlobServiceClient, ContentSettings
except Exception:
    BlobServiceClient = None
    ContentSettings = None

AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING", "")
AZURE_CONTAINER_NAME            = os.getenv("AZURE_CONTAINER_NAME", "fortifai-reports")
AZURE_BLOB_BASE_PREFIX          = os.getenv("AZURE_BLOB_BASE_PREFIX", "reports")
UPLOAD_TO_BLOB_DEFAULT = bool(AZURE_STORAGE_CONNECTION_STRING and BlobServiceClient)

_blob_client_cache = None

def _guess_content_type(path: str):
    if ContentSettings is None:
        return None
    ext = os.path.splitext(path)[1].lower()
    if ext == ".xlsx":
        return ContentSettings(content_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
    if ext == ".csv":
        return ContentSettings(content_type="text/csv")
    return ContentSettings(content_type="application/octet-stream")

def _get_blob_service_or_none():
    if not UPLOAD_TO_BLOB_DEFAULT:
        return None
    global _blob_client_cache
    if _blob_client_cache is None:
        _blob_client_cache = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
    return _blob_client_cache

def _ensure_container(name: str):
    svc = _get_blob_service_or_none()
    if svc is None: return None
    try:
        svc.create_container(name)
    except Exception:
        pass
    return svc.get_container_client(name)

def upload_folder_to_blob(local_folder: str, blob_prefix: str):
    svc = _get_blob_service_or_none()
    if svc is None:
        print(f"[Blob] Skipped upload (no credentials). Saved locally: {local_folder}")
        return
    cont = _ensure_container(AZURE_CONTAINER_NAME)
    for fname in os.listdir(local_folder):
        fpath = os.path.join(local_folder, fname)
        if os.path.isfile(fpath):
            blob_path = f"{AZURE_BLOB_BASE_PREFIX}/{blob_prefix}/{fname}".replace("\\", "/")
            with open(fpath, "rb") as fh:
                cont.upload_blob(
                    name=blob_path, data=fh, overwrite=True,
                    content_settings=_guess_content_type(fpath)
                )
    print(f"[Blob] Uploaded: {AZURE_BLOB_BASE_PREFIX}/{blob_prefix}/")

# ============================================================
# WRAPPED PERIOD RUNNER (local save + optional upload)
# ============================================================
def _run_for_period_with_optional_upload(df_full: pd.DataFrame, start_d: date, end_d: date, granularity: str, do_upload: bool):
    mask = (df_full["__report_date"].dt.date >= start_d) & (df_full["__report_date"].dt.date <= end_d)
    df_slice = df_full.loc[mask].copy()
    if df_slice.empty: return

    if granularity == "Daily":
        subfolder = f"{start_d:%Y-%m-%d}"
    elif granularity == "Weekly":
        subfolder = weekly_folder_name(start_d, end_d)
    elif granularity == "Monthly":
        subfolder = monthly_folder_name(start_d, end_d)
    else:
        subfolder = f"{start_d:%Y-%m-%d}_to_{end_d:%Y-%m-%d}"

    folder = os.path.join(BASE_OUT, granularity, subfolder)
    os.makedirs(folder, exist_ok=True)

    any_written = False

    if RUN_TOP10_VENDORS_AND_TXNS:
        p = os.path.join(folder, "Top 10 high-risk vendors by risky transactions.xlsx")
        if report_top10_vendors_and_txns(df_slice, p): any_written = True

    if RUN_HIGHRISK_TXNS:
        p = os.path.join(folder, "Top 10 high-risk transactions.xlsx")
        if report_highrisk_txns(df_slice, p): any_written = True

    if RUN_SPLIT_PO:
        p = os.path.join(folder, "Split PO Report.xlsx")
        if report_split_po(df_slice, p): any_written = True

    if RUN_PRICE_VARIANCE:
        p = os.path.join(folder, "Price Variance Risk Report.xlsx")
        if report_price_variance(df_slice, p): any_written = True

    if any_written and do_upload:
        blob_prefix = f"{granularity}/{subfolder}"
        upload_folder_to_blob(folder, blob_prefix)

# ============================================================
# PUBLIC API
# ============================================================
def _prep_df_for_reports(cleaned_df: pd.DataFrame) -> pd.DataFrame:
    dcol = choose_date_col(cleaned_df)
    df = cleaned_df.copy()
    df["__report_date"] = pd.to_datetime(df[dcol], errors="coerce")
    df = df[df["__report_date"].notna()].copy()
    return df

def run_reports_job(cleaned_df: pd.DataFrame, mode: str = "daily", *, upload_to_blob: bool | None = None):
    """
    Generate reports for the given cleaned DataFrame.
    mode: 'daily' | 'weekly' | 'monthly' | 'backfill6m'
    upload_to_blob:
        - True  -> try to upload (requires azure creds + package)
        - False -> local only
        - None  -> auto (upload if creds present, else local only)  <-- DEFAULT
    """
    df_full = _prep_df_for_reports(cleaned_df)
    if df_full.empty:
        print("No valid dates after parsing — nothing to report.")
        return

    if upload_to_blob is None:
        do_upload = UPLOAD_TO_BLOB_DEFAULT
    else:
        do_upload = bool(upload_to_blob and UPLOAD_TO_BLOB_DEFAULT)

    ensure_base_dirs()

    if mode == "daily":
        today = pd.Timestamp.now(tz=TZ_NAME).date()
        _run_for_period_with_optional_upload(df_full, today, today, "Daily", do_upload)

    elif mode == "weekly":
        today = pd.Timestamp.now(tz=TZ_NAME).date()
        start = start_of_week_monday(today)
        _run_for_period_with_optional_upload(df_full, start, today, "Weekly", do_upload)

    elif mode == "monthly":
        today = pd.Timestamp.now(tz=TZ_NAME).date()
        start = date(today.year, today.month, 1)
        _run_for_period_with_optional_upload(df_full, start, today, "Monthly", do_upload)

    elif mode == "backfill6m":
        start, end = clamp_to_six_months(df_full, "__report_date")
        for s, e in iter_daily(start, end):   _run_for_period_with_optional_upload(df_full, s, e, "Daily",   do_upload)
        for s, e in iter_weekly(start, end):  _run_for_period_with_optional_upload(df_full, s, e, "Weekly",  do_upload)
        for s, e in iter_monthly(start, end): _run_for_period_with_optional_upload(df_full, s, e, "Monthly", do_upload)

    else:
        raise ValueError("mode must be one of: daily, weekly, monthly, backfill6m")

# ============================================================
# OPTIONAL: Long-running scheduler (IST)
# ============================================================
def start_report_scheduler(get_df_callable):
    """
    Pass a zero-arg callable that returns the latest cleaned DataFrame each run.
    Example: start_report_scheduler(lambda: cleaned_df)
    """
    import pytz
    from apscheduler.schedulers.blocking import BlockingScheduler
    from apscheduler.triggers.cron import CronTrigger

    def _get_df():
        df = get_df_callable()
        if not isinstance(df, pd.DataFrame):
            raise TypeError("get_df_callable() must return a pandas DataFrame")
        return df

    def job_daily():
        print("[Scheduler] Daily job started].")
        run_reports_job(_get_df(), mode="daily")  # auto: local-only if no blob creds
        print("[Scheduler] Daily job completed.")

    def job_weekly():
        print("[Scheduler] Weekly job started.")
        run_reports_job(_get_df(), mode="weekly")
        print("[Scheduler] Weekly job completed.")

    def job_monthly():
        print("[Scheduler] Monthly job started.")
        run_reports_job(_get_df(), mode="monthly")
        print("[Scheduler] Monthly job completed.")

    tz = pytz.timezone(TZ_NAME)
    sched = BlockingScheduler(timezone=tz)

    # Daily at 02:00 IST
    sched.add_job(job_daily,  CronTrigger(day="*", hour=2, minute=0))
    # Weekly (Mondays) at 02:30 IST
    sched.add_job(job_weekly, CronTrigger(day_of_week="mon", hour=2, minute=30))
    # Monthly (1st) at 03:00 IST
    sched.add_job(job_monthly, CronTrigger(day="1", hour=3, minute=0))

    print("Report scheduler started (IST). Ctrl+C to stop.")
    try:
        sched.start()
    except (KeyboardInterrupt, SystemExit):
        print("Scheduler stopped.")

# ============================================================
# Example usage
# ============================================================
# cleaned = ...  # your DataFrame
# run_reports_job(cleaned, mode="daily")
# run_reports_job(cleaned, mode="weekly")
# run_reports_job(cleaned, mode="monthly")
run_reports_job(cleaned, mode="backfill6m")


(15007, 94)
Filtered 0 records from last 6 months out of 15007 total.


C:\Users\ajayn\AppData\Local\Temp\ipykernel_28148\1545674568.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fp_data= pd.read_sql_query(f"SELECT * FROM admin_db.{'false_positive_criteria'}", conn)


(24, 8)
Filtered: 0 rows kept (out of 0)


ValueError: No usable date column found. Set DATE_COL or add to DATE_CANDIDATES.